In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf
import tr_utils

In [2]:
importlib.reload(fma)

fm = fma.FeatureManager(target_col="trade_signal")
fm.import_trading_data(symbol="BTCUSDT",trade_timeframe="4h")
fm.import_macro_data(symbol="BTCUSDT",macro_timeframe="1d")
fm.import_super_data(symbol="BTCUSDT",super_timeframe="1w")

fm.build_features(
    lags = 1,
    macro_lags = 1,
    super_lags = 1,
    features = cf.BITCOIN_EXTERNAL_INDICATORS + cf.TRADING_TA_INDICATORS,
    macro_features = cf.MACRO_TA_INDICATORS,
    super_features = cf.SUPER_TA_INDICATORS,
)

Imported trading data from ../data/BTCUSDT-4h.csv with 11760 rows
Imported macro data from ../data/BTCUSDT-1d.csv with 1963 rows
Imported super macro data from ../data/BTCUSDT-1w.csv with 247 rows
Calculating external features ...
Calculating TA indicators of trade timeframe
Calculating TA indicators of macro timeframe
Calculating TA indicators of super timeframe
Calculating candlestick of trade timeframe
Calculating candlestick of macro timeframe
Calculating candlestick of super timeframe

Adding features for trade timeframe with lags 1: hashrate, fed_rate, gold, nasdaq, sp500, google_trend, sma_3_10, sma_7_30, sma_14_50, cci7, cci14, cci30, dx7, dx14, dx30, up_bb, low_bb, 

Adding features for macro timeframe with lags 1: sma_3_10, sma_7_30, sma_14_50, rsi7, rsi14, rsi30, cci7, cci14, cci30, dx7, dx14, dx30, up_bb, low_bb, 

Adding features for super timeframe with lags 1: sma_3_10, sma_7_30, sma_14_50, rsi7, rsi14, rsi30, cci7, cci14, cci30, dx7, dx14, dx30, up_bb, low_bb, 

Normali

In [3]:
train = fm.df.iloc[-4000:-2000]
trade = fm.df.iloc[-2000:]

In [4]:
importlib.reload(env)
importlib.reload(cf)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 10 + len(fm.cols)

train_env = env.CryptoTradingEnv(
    trade_timeframe="4h",
    df = train, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

trade_env = env.CryptoTradingEnv(
    trade_timeframe="4h",
    df = trade, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

full_env = env.CryptoTradingEnv(
    trade_timeframe="4h",
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

In [5]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

In [6]:
importlib.reload(rla)

# DQN_PARAMS = {
#     "learning_rate": 1e-4,
#     "buffer_size": 100_000,  
#     "learning_starts": 100_000,
#     "batch_size": 64,
#     "tau": 1.0,
#     "gamma": 0.9999,
#     "train_freq": 4,
#     "target_update_interval": 10000,
#     "exploration_fraction": 0.3,
#     "exploration_initial_eps": 1.0,
#     "exploration_final_eps": 0.05,
# }

# catalog_name = tr_utils.get_name_with_kwargs("dqn",DQN_PARAMS)

# dqn_model = agent.get_model(
#     model_name="dqn",
#     model_kwargs=DQN_PARAMS,
#     seed=100,
#     tensorboard_log=catalog_name
# )

PPO_PARAMS = {
    "n_steps": 8000,
    "ent_coef": 0.015,
    "learning_rate": 0.00025,
    "batch_size": 800
}

catalog_name = tr_utils.get_name_with_kwargs("no_candle",PPO_PARAMS)

ppo_model = agent.get_model(
    model_name="ppo",
    model_kwargs = PPO_PARAMS,
    tensorboard_log=catalog_name,
    seed = 100
)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
selected_model = ppo_model

selected_model = agent.train_model(
    model = selected_model,
    total_timesteps = 1_000_000,
    checkpoint = True,
    catalog_name = catalog_name,
    save_frequency = 20_000,
    progress_bar = True
)

Logging to ../logs/tensorboard_log/no_candlen_steps_8000ent_coef_0.015learning_rate_0.00025batch_size_800/PPO_0


Output()

-------------------------------------
| rollout/           |              |
|    ep_len_mean     | 7.68e+03     |
|    ep_rew_mean     | -1.94        |
| time/              |              |
|    fps             | 195          |
|    iterations      | 1            |
|    time_elapsed    | 40           |
|    total_timesteps | 8000         |
| train/             |              |
|    reward          | 0.0009696332 |
-------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.68e+03    |
|    ep_rew_mean          | -2.58       |
| time/                   |             |
|    fps                  | 210         |
|    iterations           | 2           |
|    time_elapsed         | 75          |
|    total_timesteps      | 16000       |
| train/                  |             |
|    approx_kl            | 0.007690509 |
|    clip_fraction        | 0.00667     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.0135     |
|    learning_rate        | 0.00025     |
|    loss                 | -0.016      |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.000693   |
|    reward               | -0.00020002 |
|    value_loss           | 0.00189     |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.68e+03     |
|    ep_rew_mean          | -2.41        |
| time/                   |              |
|    fps                  | 215          |
|    iterations           | 3            |
|    time_elapsed         | 111          |
|    total_timesteps      | 24000        |
| train/                  |              |
|    approx_kl            | 0.0030025195 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | 0.00071      |
|    learning_rate        | 0.00025      |
|    loss                 | -0.0196      |
|    n_updates            | 20           |
|    policy_gradient_loss | -4.28e-05    |
|    reward               | 0.005037532  |
|    value_loss           | 0.00191      |
------------------------------------------


...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.68e+03      |
|    ep_rew_mean          | -0.317        |
| time/                   |               |
|    fps                  | 235           |
|    iterations           | 71            |
|    time_elapsed         | 2409          |
|    total_timesteps      | 568000        |
| train/                  |               |
|    approx_kl            | 0.00023701454 |
|    clip_fraction        | 0.00372       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0989       |
|    explained_variance   | 9.12e-06      |
|    learning_rate        | 0.00025       |
|    loss                 | 0.000938      |
|    n_updates            | 700           |
|    policy_gradient_loss | -0.000203     |
|    reward               | 0.0           |
|    value_loss           | 0.00585       |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.68e+03      |
|    ep_rew_mean          | -0.308        |
| time/                   |               |
|    fps                  | 235           |
|    iterations           | 72            |
|    time_elapsed         | 2441          |
|    total_timesteps      | 576000        |
| train/                  |               |
|    approx_kl            | 0.00028162388 |
|    clip_fraction        | 0.00211       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.101        |
|    explained_variance   | -1.38e-05     |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0021       |
|    n_updates            | 710           |
|    policy_gradient_loss | -0.000262     |
|    reward               | 0.0           |
|    value_loss           | 0.00581       |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.68e+03      |
|    ep_rew_mean          | -0.299        |
| time/                   |               |
|    fps                  | 235           |
|    iterations           | 73            |
|    time_elapsed         | 2475          |
|    total_timesteps      | 584000        |
| train/                  |               |
|    approx_kl            | 0.00017221985 |
|    clip_fraction        | 0.00167       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.102        |
|    explained_variance   | 1.79e-06      |
|    learning_rate        | 0.00025       |
|    loss                 | 0.000633      |
|    n_updates            | 720           |
|    policy_gradient_loss | -0.000369     |
|    reward               | 0.0           |
|    value_loss           | 0.00572       |
-------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.68e+03     |
|    ep_rew_mean          | -0.289       |
| time/                   |              |
|    fps                  | 236          |
|    iterations           | 74           |
|    time_elapsed         | 2506         |
|    total_timesteps      | 592000       |
| train/                  |              |
|    approx_kl            | 0.0003995934 |
|    clip_fraction        | 0.00449      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0907      |
|    explained_variance   | 4.72e-05     |
|    learning_rate        | 0.00025      |
|    loss                 | 0.000844     |
|    n_updates            | 730          |
|    policy_gradient_loss | -0.000801    |
|    reward               | -0.00020002  |
|    value_loss           | 0.00519      |
------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.68e+03      |
|    ep_rew_mean          | -0.276        |
| time/                   |               |
|    fps                  | 236           |
|    iterations           | 75            |
|    time_elapsed         | 2539          |
|    total_timesteps      | 600000        |
| train/                  |               |
|    approx_kl            | 0.00021159471 |
|    clip_fraction        | 0.00311       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0969       |
|    explained_variance   | -0.000334     |
|    learning_rate        | 0.00025       |
|    loss                 | 0.00124       |
|    n_updates            | 740           |
|    policy_gradient_loss | -0.000197     |
|    reward               | 0.031942457   |
|    value_loss           | 0.00657       |
-------------------------------------------


In [ ]:
agent.predict(selected_model,full_env,render=True)

In [ ]:
agent.plot_multiple()

In [ ]:
state_df = pd.DataFrame(full_env.state_memory)

In [ ]:
state_df

In [ ]:
state_df.head(10)

In [ ]:
alpha = full_env.df["Close"].pct_change()

In [ ]:
alpha

In [ ]:
full_env.df

In [ ]:
state_df.head(30)